<a href="https://colab.research.google.com/github/irisroques/falhas-em-agentes/blob/main/deteccao_de_falhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção de Falhas em Sensores em Sistemas Multiagente

In [ ]:
!pip install mesa

     |████████████████████████████████| 675kB 4.1MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 


In [ ]:
#Carregando datasets
from google.colab import drive 
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd 
df=pd.read_csv('gdrive/My Drive/sensor.csv')
df.head()

,Unnamed: 0,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,0,2018-04-01 00:00:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.98958,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
1,1,2018-04-01 00:01:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.98958,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
2,2,2018-04-01 00:02:00,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,NaN,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.46875,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL
3,3,2018-04-01 00:03:00,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,NaN,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.46875,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL
4,4,2018-04-01 00:04:00,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,NaN,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.98958,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL


# Pre Processamento

In [ ]:
# Pre Processamento 
import numpy as np 
sensor = df.drop(['timestamp'], axis=1)
sensor = sensor.reset_index()
sensor.replace(np.nan, 0, inplace=True) #substituindo valores NaN por 0
sensor.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220320 entries, 0 to 220319
Data columns (total 55 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   index           220320 non-null  int64  
 1   Unnamed: 0      220320 non-null  int64  
 2   sensor_00       220320 non-null  float64
 3   sensor_01       220320 non-null  float64
 4   sensor_02       220320 non-null  float64
 5   sensor_03       220320 non-null  float64
 6   sensor_04       220320 non-null  float64
 7   sensor_05       220320 non-null  float64
 8   sensor_06       220320 non-null  float64
 9   sensor_07       220320 non-null  float64
 10  sensor_08       220320 non-null  float64
 11  sensor_09       220320 non-null  float64
 12  sensor_10       220320 non-null  float64
 13  sensor_11       220320 non-null  float64
 14  sensor_12       220320 non-null  float64
 15  sensor_13       220320 non-null  float64
 16  sensor_14       220320 non-null  float64
 17  sensor_15 

# Componente de Aprendizagem

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import seaborn as sns

X = sensor.drop('machine_status', axis=1)
y = sensor['machine_status'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)






In [ ]:
 #Gaussian Naive Bayes

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(y_pred)
print("Number of mislabeled points out of a total %d points : %d"
% (X_test.shape[0], (y_test != y_pred).sum())) # 2% de erro

['NORMAL' 'NORMAL' 'NORMAL' ... 'RECOVERING' 'NORMAL' 'NORMAL']
Number of mislabeled points out of a total 110160 points : 2275


# Tomada de Decisão

In [54]:
alvos = pd.DataFrame(y_pred)
alvos = alvos.astype(str)
alvos.rename(columns={0: 'machine_status'}, inplace=True)
teste = pd.concat([X_test,alvos])


deletados = teste.drop(teste[teste['machine_status'] == "BROKEN"].index)


O ALVO PODE TER 3 VALORES ['NORMAL', 'RECOVERING', 'BROKEN']

# Agentes

In [ ]:
import csv
import random
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid

class ConsumerAgent(Agent):

    def __init__(self, name, model, ratingarray):
        super().__init__(name, model) 
        self.name = name
        self.rate[i] = ratingarray


    def step(self):
         print("agent {}  was activated".format(self.name))
         # Whatever else the agent does when activated


class ProductEvalModel(Model):

    def __init__(self, n_agents):
       self.schedule = RandomActivation(self)
       self.grid = MultiGrid(10, 10, torus=True)
       # you could also just pass it in as a parameter, but I usually make it an 
       #attribute of the model
       self.ratingarray = [[sensor_00],[sensor_01],[sensor_02],[sensor_03],[sensor_04],[sensor_05],[sensor_06],[sensor_07],[sensor_08],[sensor_09],
                           [sensor_10],[sensor_11],[sensor_12],[sensor_13],[sensor_14],[sensor_15],[sensor_16],[sensor_17],[sensor_18],[sensor_19],[sensor_20],[sensor_21],[sensor_22],[sensor_23],[sensor_24],[sensor_25],[sensor_26],[sensor_27],[sensor_28],[sensor_29],
                           [sensor_30],[sensor_31],[sensor_32],[sensor_33],[sensor_34],[sensor_35],[sensor_36],[sensor_37],[sensor_38],[sensor_39],
                           [sensor_40],[sensor_41],[sensor_42],[sensor_43],[sensor_44],[sensor_45],[sensor_46],[sensor_47],[sensor_48],[sensor_49],
                           [sensor_50],[sensor_51],[machine_status]] 


    for i in range(n_agents):
        a = ConsumerAgent(i, self, self.ratingarray[i]) #passes in row of data
                                                        #to be agent attribute
        self.schedule.add(a)
        coords = (random.randrange(0, 10), random.randrange(0, 10))
        self.grid.place_agent(a, coords)

    def step(self):
       self.schedule.step()
       #self.dc.collect(self)

# Referencias 

Instalando Framework Mesa https://mesa.readthedocs.io/en/stable/

Base de dados disponivel em https://www.kaggle.com/nphantawee/pump-sensor-data